In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm
# from sklearn.cluster import KMeans
# #import scikit_posthocs as sp
# from scipy.stats import kruskal
# from sklearn.utils import resample
# from sklearn.feature_selection import RFE
# from catboost import CatBoostClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

pd.set_option('display.max_columns', None)  # Отображение всех столбцов
# pd.set_option('display.max_rows', None)  # Ото----бражение всех строк   

regions = pd.read_csv('regions.csv')
starts = pd.read_csv('starts.csv')
volunteers = pd.read_csv('volunteers.csv')


volunteers = volunteers[volunteers.athlete_age < 100]
starts = starts[starts.athlete_age < 80]
starts = starts[starts.finish_time < 5000]

starts['rain.1h'] = starts['rain.1h'].fillna(0)
starts['snow.1h'] = starts['snow.1h'].fillna(0)

cnt_ludei = starts.groupby(['event_name', 'event_date', 'region']).agg(
    cnt_runners = ('athlete_id', 'count')
)

ages_14 = starts[starts['athlete_age'] == 14]['athlete_id'].unique()
ages_15 = starts[starts['athlete_age'] == 15]['athlete_id'].unique()
common_athletes = set(ages_14) & set(ages_15)
starts = starts[~starts['athlete_id'].isin(common_athletes)]



starts['is_male'] = starts['athlete_gender'].apply(lambda x: 1 if x == 'мужской' else 0)
starts = starts[starts['athlete_age'] > 14]

volunteer_counts = volunteers.groupby(['event_date','event_name', 'volunteer_role_name']).size().unstack(fill_value=0)  
starts = starts.merge(volunteer_counts, on=['event_date','event_name'], how='left')  

starts = starts.dropna(subset=[col for col in starts.columns if col not in ['snow.1h', 'rain.1h']])   # удаление где 0 волонтеров в сумме было


unique_dates = (
    pd.concat([
        starts[['athlete_id', 'event_date']],
        volunteers[['athlete_id', 'event_date']]
    ])
    .drop_duplicates()
)
date_counts = unique_dates.groupby('athlete_id').size().reset_index(name='unique_event_count')
date_counts['is_repeated'] = (date_counts['unique_event_count'] > 1).astype(int)
starts = starts.merge(date_counts[['athlete_id', 'is_repeated']], on='athlete_id', how='left')


# starts = starts[~starts['region'].isin(['Республика Адыгея', 'Республика Карелия', 'Липецкая область', 'Сахалинская область', 'Архангельская область'])]  
starts = starts[starts.event_date <= '2023-11-15']


newbie = starts[starts['is_newbie'] == 1]
first_runs = newbie.sort_values(by='event_date').drop_duplicates('athlete_id')
didnt_come = first_runs[first_runs.is_repeated == 0]
come = first_runs[first_runs.is_repeated == 1]
come_with_didnt_come = pd.concat([come, didnt_come])
come_with_didnt_come['cnt_volunteers'] = come_with_didnt_come.iloc[:, 25:52].sum(axis=1)
come_with_didnt_come = come_with_didnt_come.merge(cnt_ludei,on=['event_name', 'event_date', 'region'], how="left")
print(len(come_with_didnt_come))

18114


In [8]:
come_with_didnt_come.head()

,record_id,athlete_id,event_name,event_date,city,region,finish_time,athlete_gender,athlete_age,is_newbie,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface,is_male,Ведущий велосипед,Завершение мероприятия,Замыкающий,Инструктаж новых участников,Координатор парковки,Координация волонтёров,Лидер для слабовидящих,Маршал,Обработка результатов,Организатор,Организация финиша,Пейсер,Подготовка мероприятия,Помощь в раздаче карточек позиций,Проведение разминки,Проверка карточек позиций,Проверка трассы,Раздача карточек позиций,Разное,Связи с общественностью,Секундомер,Сканирование штрих-кодов,Сортировка карточек,Составление отчёта,Сурдопереводчик,Фотограф,Хранение и доставка оборудования,is_repeated,cnt_volunteers,cnt_runners
0,4790,790069326,Елагин остров,2023-01-01,Санкт-Петербург,Санкт-Петербург,1837,женский,38,1,3.94,-1.20,996,88,2.14,100,8.00,260,Clouds,overcast clouds,0.0,0.00,смешанное,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1,15.0,41
1,4816,790126902,Петергоф Александрийский,2023-01-01,Санкт-Петербург,Санкт-Петербург,3371,женский,39,1,2.71,-1.99,997,95,1.99,100,5.95,263,Clouds,overcast clouds,0.0,0.00,грунт,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,1,20.0,55
2,4782,790126900,Волгоград панорама,2023-01-01,Волгоград,Волгоградская область,1723,женский,37,1,1.74,-3.68,1025,87,-0.16,75,7.00,260,Clouds,broken clouds,0.0,0.00,твердое,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1,13.0,30
3,4816,790126898,Петергоф Александрийский,2023-01-01,Санкт-Петербург,Санкт-Петербург,1861,женский,17,1,2.71,-1.99,997,95,1.99,100,5.95,263,Clouds,overcast clouds,0.0,0.00,грунт,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,1,20.0,55
4,4822,790126893,Раменское Городской парк,2023-01-01,Раменское,Московская область,2380,мужской,52,1,1.29,-2.06,1008,100,1.29,100,3.13,180,Drizzle,light intensity drizzle,0.0,0.22,твердое,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,1,11.0,19


In [9]:
median = come_with_didnt_come['finish_time'].median()
come_with_didnt_come1 = come_with_didnt_come[come_with_didnt_come['finish_time'] >= median]
X = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)]
y = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)]
print(len(come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей и времени больше медианы')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
# cm = confusion_matrix(y, y_pred_class)
# print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

2050


Optimization terminated successfully.
         Current function value: 0.667196
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей и времени больше медианы


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 4269
Model:                          Logit   Df Residuals:                     4241
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.02552
Time:                        10:35:33   Log-Likelihood:                -2848.3
converged:                       True   LL-Null:                       -2922.9
Covariance Type:            nonrobust   LLR p-value:                 7.152e-19
                                       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------

In [10]:
come_with_didnt_come2 = come_with_didnt_come[come_with_didnt_come['finish_time'] < median]
X = come_with_didnt_come2[(come_with_didnt_come2.cnt_runners >= 2) & (come_with_didnt_come2.cnt_runners <= 46)]
y = come_with_didnt_come2[(come_with_didnt_come2.cnt_runners >= 2) & (come_with_didnt_come2.cnt_runners <= 46)]
print(len(come_with_didnt_come2[(come_with_didnt_come2.cnt_runners >= 2) & (come_with_didnt_come2.cnt_runners <= 46)].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей и времени меньше медианы')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
# cm = confusion_matrix(y, y_pred_class)
# print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

2183
Optimization terminated successfully.
         Current function value: 0.655404
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей и времени меньше медианы


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 4569
Model:                          Logit   Df Residuals:                     4541
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.02660
Time:                        10:35:36   Log-Likelihood:                -2994.5
converged:                       True   LL-Null:                       -3076.4
Covariance Type:            nonrobust   LLR p-value:                 1.636e-21
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------

In [11]:
median = come_with_didnt_come['finish_time'].median()
come_with_didnt_come1 = come_with_didnt_come[come_with_didnt_come['finish_time'] >= median]
X = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 47) & (come_with_didnt_come1.cnt_runners <= 138)]
y = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 47) & (come_with_didnt_come1.cnt_runners <= 138)]
print(len(come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 47-138 людей и времени больше медианы')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
# cm = confusion_matrix(y, y_pred_class)
# print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

2050
Optimization terminated successfully.
         Current function value: 0.670115
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 47-138 людей и времени больше медианы


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 3120
Model:                          Logit   Df Residuals:                     3092
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.03254
Time:                        10:35:40   Log-Likelihood:                -2090.8
converged:                       True   LL-Null:                       -2161.1
Covariance Type:            nonrobust   LLR p-value:                 2.507e-17
                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------

In [12]:
come_with_didnt_come2 = come_with_didnt_come[come_with_didnt_come['finish_time'] < median]
X = come_with_didnt_come2[(come_with_didnt_come2.cnt_runners >= 47) & (come_with_didnt_come2.cnt_runners <= 138)]
y = come_with_didnt_come2[(come_with_didnt_come2.cnt_runners >= 47) & (come_with_didnt_come2.cnt_runners <= 138)]
print(len(come_with_didnt_come2[(come_with_didnt_come2.cnt_runners >= 2) & (come_with_didnt_come2.cnt_runners <= 46)].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 47-138 людей и время меньше медианы')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
# cm = confusion_matrix(y, y_pred_class)
# print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

2183
Optimization terminated successfully.
         Current function value: 0.671841
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 47-138 людей и время меньше медианы


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 3329
Model:                          Logit   Df Residuals:                     3301
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.02015
Time:                        10:35:45   Log-Likelihood:                -2236.6
converged:                       True   LL-Null:                       -2282.5
Covariance Type:            nonrobust   LLR p-value:                 5.149e-09
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------

In [13]:
median = come_with_didnt_come['finish_time'].median()
come_with_didnt_come1 = come_with_didnt_come[come_with_didnt_come['finish_time'] >= median]
X = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 139) & (come_with_didnt_come1.cnt_runners <= 404)]
y = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 139) & (come_with_didnt_come1.cnt_runners <= 404)]
print(len(come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 139-404 людей и время больше медианы')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
# cm = confusion_matrix(y, y_pred_class)
# print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

2050
Optimization terminated successfully.
         Current function value: 0.641292
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 139-404 людей и время больше медианы


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 1047
Model:                          Logit   Df Residuals:                     1019
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.06579
Time:                        10:35:47   Log-Likelihood:                -671.43
converged:                       True   LL-Null:                       -718.72
Covariance Type:            nonrobust   LLR p-value:                 1.970e-09
                                       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------

In [14]:
median = come_with_didnt_come['finish_time'].median()
come_with_didnt_come1 = come_with_didnt_come[come_with_didnt_come['finish_time'] < median]
X = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 137) & (come_with_didnt_come1.cnt_runners <= 404)]
y = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 137) & (come_with_didnt_come1.cnt_runners <= 404)]
print(len(come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 137-404 людей и времени меньше медианы')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
# cm = confusion_matrix(y, y_pred_class)
# print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

2183
Optimization terminated successfully.
         Current function value: 0.652445
         Iterations 6


Модель для предсказывания is_repeated новичка, в пробежках где было 137-404 людей и времени меньше медианы


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                  889
Model:                          Logit   Df Residuals:                      861
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.05551
Time:                        10:35:49   Log-Likelihood:                -580.02
converged:                       True   LL-Null:                       -614.11
Covariance Type:            nonrobust   LLR p-value:                 2.032e-05
                                       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------

In [15]:
def regions(region):
    arctic = [
        "Архангельская область", 
        "Республика Саха (Якутия)"
    ]

    subarctic = [
        "Республика Карелия", 
        "Республика Коми", 
        "Архангельская область", 
        "Иркутская область", 
        "Красноярский край"
    ]

    marine_climate = [
        "Калининградская область", 
        "Санкт-Петербург", 
        "Ленинградская область", 
        "Псковская область", 
        "Новгородская область"
    ]

    continental_climate = [
        "Москва", 
        "Московская область", 
        "Ярославская область", 
        "Владимирская область", 
        "Смоленская область", 
        "Калужская область", 
        "Белгородская область", 
        "Воронежская область", 
        "Липецкая область", 
        "Курская область", 
        "Тамбовская область", 
        "Тульская область", 
        "Нижегородская область", 
        "Республика Татарстан", 
        "Чувашская Республика", 
        "Республика Марий Эл", 
        "Кировская область"
    ]

    sharply_continental = [
        "Новосибирская область", 
        "Томская область", 
        "Омская область", 
        "Кемеровская область", 
        "Красноярский край", 
        "Иркутская область"
    ]

    subtropical = [
        "Краснодарский край", 
        "Республика Адыгея", 
        "Ставропольский край"
    ]

    arid_regions = [
        "Астраханская область", 
        "Волгоградская область", 
        "Ростовская область", 
        "Оренбургская область"
    ]
    highland_regions = [
        "Республика Северная Осетия — Алания"
    ]

    if region in arctic:
        return "Арктический пояс"
    elif region in subarctic:
        return 'Субарктический пояс'
    elif region in marine_climate:
        return "Умеренный пояс (Морской климат)"
    elif region in continental_climate:
        return "Умеренный пояс (Континентальный климат)"
    elif region in sharply_continental:
        return 'Умеренный пояс (Резко континентальный климат)'
    elif region in subtropical:
        return "Cубтропический климат"
    elif region in arid_regions:
        return "Засушливые районы"
    elif region in highland_regions:
        return "Высокогорные районы"

In [16]:
come_with_didnt_come['zone'] = come_with_didnt_come['region'].apply(regions)
come_with_didnt_come.head()

,record_id,athlete_id,event_name,event_date,city,region,finish_time,athlete_gender,athlete_age,is_newbie,temp,feels_like,pressure,humidity,dew_point,clouds,wind_speed,wind_deg,weather.main,weather.description,rain.1h,snow.1h,track_description_surface,is_male,Ведущий велосипед,Завершение мероприятия,Замыкающий,Инструктаж новых участников,Координатор парковки,Координация волонтёров,Лидер для слабовидящих,Маршал,Обработка результатов,Организатор,Организация финиша,Пейсер,Подготовка мероприятия,Помощь в раздаче карточек позиций,Проведение разминки,Проверка карточек позиций,Проверка трассы,Раздача карточек позиций,Разное,Связи с общественностью,Секундомер,Сканирование штрих-кодов,Сортировка карточек,Составление отчёта,Сурдопереводчик,Фотограф,Хранение и доставка оборудования,is_repeated,cnt_volunteers,cnt_runners,zone
0,4790,790069326,Елагин остров,2023-01-01,Санкт-Петербург,Санкт-Петербург,1837,женский,38,1,3.94,-1.20,996,88,2.14,100,8.00,260,Clouds,overcast clouds,0.0,0.00,смешанное,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,1,15.0,41,Умеренный пояс (Морской климат)
1,4816,790126902,Петергоф Александрийский,2023-01-01,Санкт-Петербург,Санкт-Петербург,3371,женский,39,1,2.71,-1.99,997,95,1.99,100,5.95,263,Clouds,overcast clouds,0.0,0.00,грунт,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,1,20.0,55,Умеренный пояс (Морской климат)
2,4782,790126900,Волгоград панорама,2023-01-01,Волгоград,Волгоградская область,1723,женский,37,1,1.74,-3.68,1025,87,-0.16,75,7.00,260,Clouds,broken clouds,0.0,0.00,твердое,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1,13.0,30,Засушливые районы
3,4816,790126898,Петергоф Александрийский,2023-01-01,Санкт-Петербург,Санкт-Петербург,1861,женский,17,1,2.71,-1.99,997,95,1.99,100,5.95,263,Clouds,overcast clouds,0.0,0.00,грунт,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,3.0,0.0,1,20.0,55,Умеренный пояс (Морской климат)
4,4822,790126893,Раменское Городской парк,2023-01-01,Раменское,Московская область,2380,мужской,52,1,1.29,-2.06,1008,100,1.29,100,3.13,180,Drizzle,light intensity drizzle,0.0,0.22,твердое,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,1,11.0,19,Умеренный пояс (Континентальный климат)


In [17]:
for zone in come_with_didnt_come['zone']:
    come_with_didnt_come1 = come_with_didnt_come[come_with_didnt_come['zone'] == zone]
    X = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)]
    y = come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)]
    print(len(come_with_didnt_come1[(come_with_didnt_come1.cnt_runners >= 2) & (come_with_didnt_come1.cnt_runners <= 46)].drop_duplicates(['event_name', 'event_date'])))


    need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                    'Ведущий велосипед', 
                    'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                    'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
                'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                    'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                    'Фотограф', 'Хранение и доставка оборудования',
                    'cnt_runners'
                    ]
                    






    for col1 in need_columns:
        for col2 in need_columns:
            if col1 != col2:
                try:
                    coef = X[col1].corr(X[col2],method='spearman')
                    if abs(coef) > 0.8:
                        print('Возможна мультиколлинеарность: ', coef, col1, col2)
                except:
                    print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

    for_model = X[need_columns]


    for_model = sm.add_constant(for_model)
    y = y['is_repeated']
    model = sm.Logit(y, for_model).fit()
    print('\n')
    print(f'Модель для предсказывания is_repeated новичка, в пробежках где было 1-46 людей и климатическая зона: {zone}')
    print('\n')
    print(model.summary())
    marginal_effects = model.get_margeff()
    print('\n')
    print('Предельные эффекты:')
    print(marginal_effects.summary())
    print('\n')

    y_pred = model.predict(for_model)
    y_pred_class = (y_pred > 0.5).astype(int)  
    # cm = confusion_matrix(y, y_pred_class)
    # print(cm)
    accuracy = (y_pred_class == y).mean()  
    print(f'Accuracy: {accuracy:.2f}')  

311
         Current function value: 0.647630
         Iterations: 35


Модель для предсказывания is_repeated новичка, в пробежках где было 1-46 людей и климатическая зона: Умеренный пояс (Морской климат)


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                  975
Model:                          Logit   Df Residuals:                      947
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.03465
Time:                        10:35:51   Log-Likelihood:                -631.44
converged:                      False   LL-Null:                       -654.11
Covariance Type:            nonrobust   LLR p-value:                   0.01500
                                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------

/Users/aleksejgavrikov/abort-404/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.647630
         Iterations: 35


Модель для предсказывания is_repeated новичка, в пробежках где было 1-46 людей и климатическая зона: Умеренный пояс (Морской климат)


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                  975
Model:                          Logit   Df Residuals:                      947
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.03465
Time:                        10:35:54   Log-Likelihood:                -631.44
converged:                      False   LL-Null:                       -654.11
Covariance Type:            nonrobust   LLR p-value:                   0.01500
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------

/Users/aleksejgavrikov/abort-404/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.638297
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 1-46 людей и климатическая зона: Засушливые районы


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                  681
Model:                          Logit   Df Residuals:                      653
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.04006
Time:                        10:35:56   Log-Likelihood:                -434.68
converged:                       True   LL-Null:                       -452.82
Covariance Type:            nonrobust   LLR p-value:                    0.1093
                                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------

...

Optimization terminated successfully.
         Current function value: 0.658632
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 1-46 людей и климатическая зона: Умеренный пояс (Континентальный климат)


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 4677
Model:                          Logit   Df Residuals:                     4649
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.03050
Time:                        10:35:59   Log-Likelihood:                -3080.4
converged:                       True   LL-Null:                       -3177.3
Covariance Type:            nonrobust   LLR p-value:                 3.663e-27
                                       coef    std err          z      P>|z|      [0.025      0.975]
--------------------

/Users/aleksejgavrikov/abort-404/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: 0.647630
         Iterations: 35


Модель для предсказывания is_repeated новичка, в пробежках где было 1-46 людей и климатическая зона: Умеренный пояс (Морской климат)


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                  975
Model:                          Logit   Df Residuals:                      947
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.03465
Time:                        10:36:02   Log-Likelihood:                -631.44
converged:                      False   LL-Null:                       -654.11
Covariance Type:            nonrobust   LLR p-value:                   0.01500
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------

/Users/aleksejgavrikov/abort-404/venv/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


ValueError: zero-size array to reduction operation maximum which has no identity